# object ray intersection classes

In [1]:
# setup stuff
import sys
sys.path.append("c:\\Users\\Dave_Semeraro\\Documents\\TeXRay")
from txr.render import rays, image
import matplotlib.pyplot as plt 
from PIL import Image
%matplotlib inline
import numpy as np

# interacting with data
We need a class that can handle interaction between rays and data. Lets call this class Target. A target will accept a group of rays and "interact" the rays with the data in the target. The sort of interaction is undefined at present. 

At this point Im not certain how to do this. The form of the data is ambiguous. It could for example be particle data or scalar data on some sort of mesh. The method to apply to the data is also undefined it could be a sampling or intersection operation. Finally the returned value structure is unknown. So we know nothing. 

Would it be clever to define a class and then derive subclasses from it or decorate the class instead. 

In [2]:
class target:
    """ 
    A target is a wrapper around data that will interact
    rays with the data in a specified way. 

    """
    def __init__(self,data):
        # keep the data handle as we may use this 
        # target again with other rays 
        self._data = data
        self._method = None 
        self._rayGroup = None

    @property
    def method(self):
        return self._method

    @method.setter
    def method(self,value):
        self._method = value

    @property
    def rayGroup(self):
        return self._rayGroup

    @rayGroup.setter
    def rayGroup(self,value):
        self._rayGroup = value

    def apply_method_to_data(self):
        assert(self._method is not None)
        assert(self._rayGroup is not None)
        return(self._method(self._data,self._rayGroup))

In [3]:
class sphere:

    def __init__(self,center,radius):
        self._radius = radius
        self._center = center[:]

    @property
    def radius(self):
        return self._radius

    @property 
    def center(self):
        return self._center

def hit_sphere(sphere,rayGroup):
    raygrouparray = rayGroup.rays
    rows = raygrouparray.shape[0]
    for r in range(rows):
        origin = raygrouparray[r,0:3]
        direction = raygrouparray[r,3:6]
        oc = origin - sphere.center
        a = np.dot(direction,direction)
        b = 2.0*np.dot(oc,direction)
        c = np.dot(oc,oc) - sphere.radius*sphere.radius 
        disc = b*b -4.0*a*c
        if disc > 0:
            raygrouparray[r,9:12] = (255,0,0)
        else:
            raygrouparray[r,9:12] = (0,0,255)

In [7]:
# lets create a target with the data being the sphere
# sphere first
center = np.array((0.,0.,-1.))
rad = 0.5
marble = sphere(center,rad)
bullseye = target(marble)
bullseye.method = hit_sphere
# now we need a ray or two. 
# one ray for now. 
origin = [np.array((0.0,0.0,0.0),dtype=np.float)]
direction = [np.array((0.,0.,-1.),dtype=np.float)]
# rg contains the ray group
rgen = rays.ray_generator(origin,direction)
rgen.generate()
rg = rgen.ray_group
# now lets assign the raygroup to the target
bullseye.rayGroup = rg


In [8]:
# now I have a target and stuff lets fire it off. 
print(rg[:])

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00 -1.0000000e+00  1.1754944e-38  3.4028235e+38
   1.1754944e-38  0.0000000e+00  0.0000000e+00  0.0000000e+00]]


In [9]:
bullseye.apply_method_to_data()
print(rg[:])

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00 -1.0000000e+00  1.1754944e-38  3.4028235e+38
   1.1754944e-38  2.5500000e+02  0.0000000e+00  0.0000000e+00]]


The exercise above seems to have worked for the simple case of the single ray hitting the sphere. The target method modifies the data in the ray group's data matrix and the change in the payload persists. This model seems to work. 